In [ ]:
from multiprocessing import Process, Queue
import subprocess

from jakenode.database_connector import run_query

WORKFLOW_ID = 1

def run_xpra_window(queue, workflow_id=WORKFLOW_ID):
    script_target = 'run_two_window_app.py --account-id 1 --workflow-category-id 1'
    x11_screen = 80
    xpra_target_port = '8080'
    user = 'myuser'
    
    current_working_directory = subprocess.check_output('pwd')
    current_working_directory = current_working_directory.decode().strip()
    script_filepath = f'{current_working_directory}/{script_target}'

    base_command = f'xpra start :{x11_screen}'
    flags = [
        f'--bind-tcp=0.0.0.0:{xpra_target_port}',
        '--mdns=no',
        '--webcam=no',
        '--no-daemon',
        '--pulseaudio=no',
        '--min-quality=90',
        f'--start="python3 {script_filepath}"'
    ]
    
    flags = ' '.join(flags)
     
    command = f"su myuser -c '{base_command} {flags}'"
    print(command)
    subprocess.call(command, shell=True)

run_xpra_window(None)

In [ ]:
!killall -u myuser


In [ ]:
from itertools import chain

from jakenode.database_connector import run_query

def invert_address_types(address_types):
    column_types = {}
    for address_type, column_list in address_types.items():
        for column_name in column_list:
            column_types[column_name] = address_type
    return column_types

def generate_fetch_query(columns, destination_server):
    address_select_clause = f",\n            ".join(columns)

    return f"""
        SELECT
            {address_select_clause}
        FROM workflow_routes
        WHERE
            app_server_address='{destination_server}'
    """
    
def fetch_taken_addresses(column_types, destination_server):
            
    query = generate_fetch_query(column_types.keys(), destination_server)
    
    taken_addresses = run_query(query, return_data_format=dict)
    filtered_taken_addresses = {}
    for column_name, address_type in column_types.items():
        filtered_taken_addresses.setdefault(address_type, [])
        filtered_taken_addresses[address_type] += [i for i in taken_addresses[column_name] if isinstance(i, int)]

    return filtered_taken_addresses

def assign_next_port(workflow_id):
    DESTINATION_SERVER = 'localhost'
    
    address_search_ranges = {
        'ports': (49152, 65535),
        'websockets': (49152, 65535),
        'x11_displays': (0, 99)
    }
    
    address_types = {
        'ports': ['xpra_port', 'info_panel_port'],
        'websockets': ['websocket'],
        'x11_displays': ['x11_display']
    }
    
    column_types = invert_address_types(address_types)

    workflow_addresses = run_query(
        f"""
            {generate_fetch_query(chain(*address_types.values()), DESTINATION_SERVER)}
            AND workflow_id={workflow_id}
        """,
        return_data_format=dict
    )
    
    taken_address_dict = fetch_taken_addresses(column_types, DESTINATION_SERVER)
    update_column_dict = {}

    for column_name in column_types:
        if not workflow_addresses[column_name][0]:
            address_type = column_types[column_name]
            taken_addresses = taken_address_dict[address_type]
            current_address, max_allowable_address = address_search_ranges[address_type]
            assigned_address = None
            
            max_value_err_msg = f'Cannot assign {address_type}.'
            max_value_err_msg += f'All values {current_address}-{max_allowable_address} already taken.'
            
            while assigned_address is None:
                if current_address > max_allowable_address:
                    raise Exception(max_value_err_msg)
                elif current_address not in taken_addresses:
                    assigned_address = current_address
                else:
                    current_address += 1
            update_column_dict[column_name] = assigned_address
            

    if update_column_dict:
        set_clause = []
        for column_name, set_value in update_column_dict.items():
            set_clause.append(f'{column_name}={set_value}')
        set_clause = ',\n            '.join(set_clause)
        update_statement = f"""
            UPDATE workflow_routes
            SET
                {set_clause}
            ;
        """
        print(update_statement)
        run_query(update_statement, commit=True)

assign_next_port(3)